In [1]:
import tensorflow as tf

# Ensure eager execution is enabled
tf.config.run_functions_eagerly(True)

# Optional: reset any previous session
tf.keras.backend.clear_session()


In [1]:
import os


In [2]:
os.chdir(r"C:\Users\Asus\Desktop\kidney_Disease_Classifier")

In [3]:
%pwd

'C:\\Users\\Asus\\Desktop\\kidney_Disease_Classifier'

In [4]:
import os
from pathlib import Path
from dataclasses import dataclass
from typing import Tuple
import yaml
import tensorflow as tf

In [5]:
# config_entity.py
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    
    # Core training params
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: List[int]       # matches IMAGE_SIZE in params.yaml
    params_learning_rate: float        # matches LEARNING_RATE
    params_classes: int                # matches CLASSES
    params_freeze_till: int            # matches FREEZE_TILL
    params_fine_tune_last_n: int       # matches FINE_TUNE_LAST_N
    
    # Regularization (anti-overfitting)
    params_dropout_rate_head: float    # matches DROPOUT_RATE_HEAD
    params_weight_decay: float         # matches WEIGHT_DECAY
    params_label_smoothing: float      # matches LABEL_SMOOTHING
    
    # Model architecture
    params_dense_units: int            # matches DENSE_UNITS
    params_optimizer: str              # matches OPTIMIZER


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common  import read_yaml, create_directories
import tensorflow as tf

In [7]:
from cnnClassifier.entity.config_entity import (
    DataIngestionConfig,
    PrepareBaseModelConfig,
    TrainingConfig
)



In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config['training']
        prepare_base_model = self.config['prepare_base_model']
        params = self.params

        training_data = os.path.join(
            self.config['data_ingestion']['unzip_dir'],
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
        )

        create_directories([Path(training['root_dir'])])

        
        return TrainingConfig(
        root_dir=Path(training['root_dir']),
        trained_model_path=Path(training['trained_model_path']),
        updated_base_model_path=Path(prepare_base_model['updated_base_model_path']),
        training_data=Path(training_data),

        # --- Core training params ---
        params_epochs=params.get('EPOCHS', 25),
        params_batch_size=params.get('BATCH_SIZE', 16),
        params_is_augmentation=params.get('AUGMENTATION', True),
        params_image_size=params.get('IMAGE_SIZE', [224, 224, 3]),
        params_learning_rate=params.get('LEARNING_RATE', 1e-4),
        params_classes=params.get('CLASSES', 4),
        params_freeze_till=params.get('FREEZE_TILL', 120),
        params_fine_tune_last_n=params.get('FINE_TUNE_LAST_N', 60),

        # --- Regularization ---
        params_dropout_rate_head=params.get('DROPOUT_RATE_HEAD', 0.4),
        params_weight_decay=params.get('WEIGHT_DECAY', 0.0002),
        params_label_smoothing=params.get('LABEL_SMOOTHING', 0.1),

        # --- Model architecture tweaks ---
        params_dense_units=params.get('DENSE_UNITS', 512),
        params_optimizer=params.get('OPTIMIZER', "adamw"),
)



In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
import tensorflow as tf
from pathlib import Path
from cnnClassifier.entity.config_entity import TrainingConfig
from tensorflow.keras.optimizers import AdamW


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model: tf.keras.Model | None = None
        self.train_generator: tf.keras.preprocessing.image.DirectoryIterator | None = None
        self.valid_generator: tf.keras.preprocessing.image.DirectoryIterator | None = None

    # -----------------------
    # Load base model
    # -----------------------
    def get_base_model(self) -> tf.keras.Model:
        path = str(self.config.updated_base_model_path)
        if not path.endswith(".keras") and not path.endswith(".h5"):
            path += ".keras"
        self.model = tf.keras.models.load_model(path)
        print(f"✅ Model loaded from: {path}")
        return self.model

    # -----------------------
    # Unfreeze last N layers
    # -----------------------
    def unfreeze_top_layers(self, num_layers: int = None, learning_rate: float = None):
        if self.model is None:
            raise ValueError("Load the model first using get_base_model()")

        if num_layers is None:
            num_layers = self.config.params_fine_tune_last_n
        if learning_rate is None:
            learning_rate = self.config.params_learning_rate

        # Unfreeze selected layers
        for layer in self.model.layers[-num_layers:]:
            layer.trainable = True

        print(f"✅ Top {num_layers} layers set to trainable for fine-tuning.")

        # Recompile model
        self.model.compile(
            optimizer=AdamW(
                learning_rate=learning_rate,
                weight_decay=self.config.params_weight_decay
            ),
            loss=tf.keras.losses.CategoricalCrossentropy(
                label_smoothing=self.config.params_label_smoothing
            ),
            metrics=[
                "accuracy",
                tf.keras.metrics.AUC(name="auc"),
                tf.keras.metrics.Precision(name="precision"),
                tf.keras.metrics.Recall(name="recall")
            ]
        )
        print(f"✅ Model recompiled after unfreezing layers (lr={learning_rate}).")

    # -----------------------
    # Generators
    # -----------------------
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.2,
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:2],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"
        )

        # Validation generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            str(self.config.training_data),
            subset="validation",
            shuffle=False,
            seed=42,
            **dataflow_kwargs
        )

        # Training generator with augmentation
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1.0 / 255,
                rotation_range=20,
                zoom_range=0.2,
                horizontal_flip=True,
                vertical_flip=False,
                brightness_range=[0.8, 1.2],
                fill_mode="nearest",
                validation_split=0.2
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            str(self.config.training_data),
            subset="training",
            shuffle=True,
            seed=42,
            **dataflow_kwargs
        )

    # -----------------------
    # Callbacks
    # -----------------------
    def get_callbacks(self):
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=5,
            restore_best_weights=True,
            mode="max",
            verbose=1
        )
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_accuracy",
            factor=0.2,
            patience=3,
            verbose=1,
            min_lr=1e-7
        )
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.trained_model_path),
            save_best_only=True,
            monitor="val_accuracy",
            mode="max",
            verbose=1
        )
        return [early_stopping, reduce_lr, checkpoint]

    # -----------------------
    # Training with progressive unfreezing
    # -----------------------
    def train(self):
        if self.model is None:
            raise ValueError("Model not loaded. Call get_base_model() first.")
        if self.train_generator is None or self.valid_generator is None:
            raise ValueError("Generators not prepared. Call train_valid_generator() first.")

        callbacks = self.get_callbacks()

        # Progressive unfreezing schedule (layers, learning_rate)
        unfreeze_schedule = [
            (10, 1e-4),   # Phase 1
            (20, 5e-5),   # Phase 2
            
        ]

        history = None
        for i, (layers, lr) in enumerate(unfreeze_schedule, 1):
            print(f"\n🚀 Phase {i}: Unfreezing last {layers} layers with lr={lr}...")
            self.unfreeze_top_layers(num_layers=layers, learning_rate=lr)
            history = self.model.fit(
                self.train_generator,
                validation_data=self.valid_generator,
                epochs=5,  # per phase
                callbacks=callbacks
            )

        # Save final model (best checkpoint already saved)
        self.save_model(self.config.trained_model_path, self.model)
        return history

    # -----------------------
    # Save model
    # -----------------------
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        path_str = str(path)
        if not path_str.endswith(".keras") and not path_str.endswith(".h5"):
            path_str += ".keras"
        model.save(path_str, include_optimizer=False)
        print(f"✅ Model saved at: {path_str}")


In [16]:

try:
    # 1️⃣ Load configuration
    config = ConfigurationManager()
    training_config = config.get_training_config()

    # 2️⃣ Initialize Training
    training = Training(config=training_config)

    # 3️⃣ Load the updated base model (with custom head)
    training.get_base_model()

    # 4️⃣ (Optional) Unfreeze last N layers for fine-tuning
    training.unfreeze_top_layers(num_layers=training_config.params_fine_tune_last_n)

    # 5️⃣ Prepare training and validation generators
    training.train_valid_generator()

    # 6️⃣ Train the model with callbacks (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)
    history = training.train()

    # 7️⃣ Save the final fine-tuned model
    training.save_model(training_config.trained_model_path, training.model)

except Exception as e:
    raise e


[2025-09-20 14:48:42,213: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-20 14:48:42,216: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-20 14:48:42,219: INFO: common: created directory at: artifacts_root]
[2025-09-20 14:48:42,220: INFO: common: created directory at: artifacts\training]
✅ Model loaded from: artifacts\prepare_base_model\updated_base_model.keras
✅ Top 40 layers set to trainable for fine-tuning.
✅ Model recompiled after unfreezing layers (lr=5e-05).
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.

🚀 Phase 1: Unfreezing last 10 layers with lr=0.0001...
✅ Top 10 layers set to trainable for fine-tuning.
✅ Model recompiled after unfreezing layers (lr=0.0001).


c:\Users\Asus\anaconda3\envs\kidney\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
623/623 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7429 - auc: 0.9127 - loss: 0.9892 - precision: 0.8595 - recall: 0.5942
Epoch 1: val_accuracy improved from None to 0.80901, saving model to artifacts\training\model.keras
623/623 ━━━━━━━━━━━━━━━━━━━━ 968s 2s/step - accuracy: 0.8738 - auc: 0.9805 - loss: 0.7765 - precision: 0.9390 - recall: 0.8039 - val_accuracy: 0.8090 - val_auc: 0.9453 - val_loss: 0.9243 - val_precision: 0.8274 - val_recall: 0.7901 - learning_rate: 1.0000e-04
Epoch 2/5
623/623 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9818 - auc: 0.9991 - loss: 0.5767 - precision: 0.9884 - recall: 0.9711
Epoch 2: val_accuracy did not improve from 0.80901
623/623 ━━━━━━━━━━━━━━━━━━━━ 978s 2s/step - accuracy: 0.9840 - auc: 0.9993 - loss: 0.5635 - precision: 0.9894 - recall: 0.9763 - val_accuracy: 0.7415 - val_auc: 0.9025 - val_loss: 1.1070 - val_precision: 0.7507 - val_recall: 0.7266 - learning_rate: 1.0000e-04
Epoch 3/5
623/623 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accurac